In [37]:
import requests
import pandas as pd

# No Matchups Yet
# league_id = 781936337963036672

# Redraft Last Place
league_id = 716027744466583552
filename = "Redraft_Matchups.xlsx"

In [38]:
# Fetch NFL Players
URL = "https://api.sleeper.app/v1/players/nfl"
response = requests.get(URL)
player_dictionary = response.json()

# Create lookup dictionaries for roster_id to user_id and roster_id to display_name
URL = f"https://api.sleeper.app/v1/league/{league_id}/rosters"
response = requests.get(URL)
rosters = response.json()
roster_id2user_id = {roster["roster_id"]: roster["owner_id"] for roster in rosters}

URL = f"https://api.sleeper.app/v1/league/{league_id}/users"
response = requests.get(URL)
users = response.json()
user_id2display_name = {user["user_id"]: user["display_name"] for user in users}

roster_id2display_name = {roster["roster_id"]: user_id2display_name[roster["owner_id"]] for roster in rosters}

# Get players used in each matchup and the points they scored

columns = ["week", "matchup_id", "roster_id", "display_name", "player_id", "player", "starter", "points"]
df = []

# Find Max Weeks
week = 1
URL = f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week}"
response = requests.get(URL)
matchup_rosters = response.json()
while len(matchup_rosters) != 0:
    for roster in matchup_rosters:
        matchup_id = roster["matchup_id"]
        roster_id = roster["roster_id"]
        display_name = roster_id2display_name[roster_id]
        starters = roster['starters']
        for player_id, points in roster["players_points"].items():
            # IKE: USE THIS TO GET PLAYERS TO MATCH UP WITH ROSTERS
            player = player_dictionary[player_id]["first_name"] + " " + player_dictionary[player_id]["last_name"]
            if player_id in starters:
                starter = 'Y'
            else:
                starter = 'N'
            new_row = [week, matchup_id, roster_id, display_name, player_id, player, starter, points]
            df.append(new_row)
    week += 1
    URL = f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week}"
    response = requests.get(URL)
    matchup_rosters = response.json()

matchup_df = pd.DataFrame(df, columns = columns)
matchup_df.to_excel(f"../Data/{filename}", index = False)
matchup_df

,week,matchup_id,roster_id,display_name,player_id,player,starter,points
0,1,1.0,1,keaton22,TB,Tampa Bay Buccaneers,Y,-2.0
1,1,1.0,1,keaton22,947,Julio Jones,Y,4.4
2,1,1.0,1,keaton22,7593,Trey Sermon,N,0.0
3,1,1.0,1,keaton22,7571,Rashod Bateman,N,0.0
4,1,1.0,1,keaton22,7553,Kyle Pitts,Y,5.1
...,...,...,...,...,...,...,...,...
3736,18,NaN,12,KingDarsh,367,Jared Cook,N,0.0
3737,18,NaN,12,KingDarsh,3157,Will Fuller,N,0.0
3738,18,NaN,12,KingDarsh,2319,DeVante Parker,N,0.0
3739,18,NaN,12,KingDarsh,184,Adrian Peterson,N,0.0
